# Day 4 - LangChain

In [ ]:
%pip install langchain_community -q

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,  # Slightly higher for more conversational responses
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Financial Coach System Prompt
system_prompt = """You are an expert financial coach with deep knowledge in:
- Personal budgeting and expense management
- Investment strategies (stocks, bonds, mutual funds, ETFs)
- Retirement planning (401k, IRA, pension plans)
- Debt management and credit improvement
- Tax optimization strategies
- Emergency fund planning
- Financial goal setting

Your role is to:
1. Provide personalized financial advice based on user situations
2. Educate users on financial concepts in simple terms
3. Help users create actionable financial plans
4. Be supportive and non-judgmental
5. Always include disclaimers that this is educational advice, not professional financial planning

Guidelines:
- Ask clarifying questions when needed
- Provide specific, actionable recommendations
- Use examples to illustrate concepts
- Be encouraging about financial progress
- Remind users to consult certified financial advisors for major decisions"""

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{user_question}")
])

# Create output parser
output_parser = StrOutputParser()

# Build the financial coaching chain
financial_coach_chain = (
    {"user_question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

print("=" * 60)
print("FINANCIAL COACHING AGENT - LangChain Version")
print("=" * 60)

# Scenario 1: Budget Planning
print("\n📊 SCENARIO 1: Budget Planning")
print("-" * 60)
question1 = """I'm 28 years old, earning $75,000/year, and have $5,000 in savings. 
I want to start budgeting better. Where should I start?"""

response1 = financial_coach_chain.invoke(question1)
print(f"Question: {question1}")
print(f"\nCoach Response:\n{response1}")

# Scenario 2: Investment Advice
print("\n" + "=" * 60)
print("💰 SCENARIO 2: Investment Advice")
print("-" * 60)
question2 = """I have $10,000 to invest and I'm 30 years old. 
Should I invest in index funds or individual stocks?"""

response2 = financial_coach_chain.invoke(question2)
print(f"Question: {question2}")
print(f"\nCoach Response:\n{response2}")

# Scenario 3: Debt Management (Streaming)
print("\n" + "=" * 60)
print("💳 SCENARIO 3: Debt Management (Streaming)")
print("-" * 60)
question3 = """I have $15,000 in credit card debt across 3 cards with interest rates 
between 18-24%. What's the best strategy to pay this off?"""

print(f"Question: {question3}")
print("\nCoach Response (streaming):")
for chunk in financial_coach_chain.stream(question3):
    print(chunk, end="", flush=True)
print("\n")

# Scenario 4: Batch Questions
print("=" * 60)
print("📚 SCENARIO 4: Quick Financial Tips (Batch)")
print("-" * 60)

quick_questions = [
    "What's the 50/30/20 budget rule?",
    "How much should I have in my emergency fund?",
    "What's the difference between a Roth IRA and Traditional IRA?"
]

batch_responses = financial_coach_chain.batch(quick_questions)

for i, (q, r) in enumerate(zip(quick_questions, batch_responses), 1):
    print(f"\nQ{i}: {q}")
    print(f"A{i}: {r[:200]}...")  # Truncate for readability

print("\n" + "=" * 60)
print("Session Complete!")
print("=" * 60)